# Homework 4

These following code shows which packages are used.

In [25]:
library(stringr)
library(dplyr)
library(tidyr)
library(zoo)

## Joining Data

These are the dowloaded files.

In [22]:
df_cellphones <- read.csv("https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/cell_phones_total.csv")
df_population <- read.csv("https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/pop_data.csv")
df_country <- read.csv("https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/country_data.csv")

In this task, we combined cleaned cell phone and population data to analyze phones per capita. Using the convert_to_numeric function and forward fill from the previous exercise, we handled missing values and reshaped both datasets. After merging them, we created a table with the columns iso.3, year, n_cellphones, and population. The final table resembles the one from the homework but may differ slightly due to our approach to handling missing values.

In [34]:
convert_to_numeric <- function(value) {
  if (is.character(value)) {
    value <- trimws(value)
    multipliers <- c(k = 1e3, M = 1e6, B = 1e9)
    for (suffix in names(multipliers)) {
      if (grepl(suffix, value)) {
        return(as.numeric(gsub(suffix, "", value)) * multipliers[suffix])
      }
    }
    return(as.numeric(value))
  }
  return(value)
}
#Converting values with suffixes like k, M, B into numeric


df_cellphones[, -1] <- lapply(df_cellphones[, -1], function(x) sapply(x, convert_to_numeric))


df_cellphones[] <- lapply(df_cellphones, function(x) na.locf(x, na.rm = FALSE))  
df_cellphones[is.na(df_cellphones)] <- 0
#Forward fill
#Replace any remaining NAs with 0

df_cellphones_clean <- df_cellphones %>%
  pivot_longer(cols = starts_with("X"), names_to = "year", values_to = "n_cellphones") %>%
  mutate(year = as.numeric(gsub("X", "", year)))
#Reshaping df_cellphones into a long format and cleaning up the year column


df_population[] <- lapply(df_population, function(x) na.locf(x, na.rm = FALSE)) 
df_population[is.na(df_population)] <- 0  
#Forward fill population data
#Replace any remaining NAs with 0


df_population_clean <- df_population %>%
  pivot_longer(cols = starts_with("X"), names_to = "year", values_to = "population") %>%
  mutate(year = as.numeric(gsub("X", "", year)))
#Reshaping the df_population to a long format and cleaning up the year column

                          
df_population_clean <- df_population_clean %>%
  rename(iso_3 = iso.3)  
#Rename iso.3 to iso_3 for consistency


df_combined <- df_cellphones_clean %>%
  left_join(df_population_clean, by = c("iso.3" = "iso_3", "year" = "year"))


df_combined <- df_combined %>%
  mutate(phones_per_capita = n_cellphones / population)
#For later exercise


final_table <- df_combined %>%
  select(iso.3, year, n_cellphones, population,)
#Relevant columns selected


head(final_table)

iso.3,year,n_cellphones,population
<chr>,<dbl>,<dbl>,<dbl>
ABW,1960,0,54608
ABW,1965,0,58782
ABW,1966,0,59291
ABW,1967,0,59522
ABW,1968,0,59471
ABW,1969,0,59330


In [26]:
colnames(df_country)
colnames(df_cellphones)
colnames(df_population)
#Column names are checked because i did something wrong

[1] "name"                     "alpha.2"                 
 [3] "alpha.3"                  "country.code"            
 [5] "iso_3166.2"               "region"                  
 [7] "sub.region"               "intermediate.region"     
 [9] "region.code"              "sub.region.code"         
[11] "intermediate.region.code"

[1] "iso.3" "X1960" "X1965" "X1966" "X1967" "X1968" "X1969" "X1970" "X1971"
[10] "X1972" "X1973" "X1974" "X1975" "X1976" "X1977" "X1978" "X1979" "X1980"
[19] "X1981" "X1982" "X1983" "X1984" "X1985" "X1986" "X1987" "X1988" "X1989"
[28] "X1990" "X1991" "X1992" "X1993" "X1994" "X1995" "X1996" "X1997" "X1998"
[37] "X1999" "X2000" "X2001" "X2002" "X2003" "X2004" "X2005" "X2006" "X2007"
[46] "X2008" "X2009" "X2010" "X2011" "X2012" "X2013" "X2014" "X2015" "X2016"
[55] "X2017" "X2018" "X2019"

[1] "X"     "iso.3" "X1960" "X1961" "X1962" "X1963" "X1964" "X1965" "X1966"
[10] "X1967" "X1968" "X1969" "X1970" "X1971" "X1972" "X1973" "X1974" "X1975"
[19] "X1976" "X1977" "X1978" "X1979" "X1980" "X1981" "X1982" "X1983" "X1984"
[28] "X1985" "X1986" "X1987" "X1988" "X1989" "X1990" "X1991" "X1992" "X1993"
[37] "X1994" "X1995" "X1996" "X1997" "X1998" "X1999" "X2000" "X2001" "X2002"
[46] "X2003" "X2004" "X2005" "X2006" "X2007" "X2008" "X2009" "X2010" "X2011"
[55] "X2012" "X2013" "X2014" "X2015" "X2016" "X2017" "X2018" "X2019" "X2020"

For a better presentation of the table, we replaced ISO-3 codes with country names using a mapping from country_data.csv. We combined the cell phone, population, and country data into a single table, calculating phones per capita (n_cellphones/population).

In [35]:
df_combined <- df_cellphones_clean %>%
  left_join(df_population_clean, by = c("iso_3", "year")) %>%
  left_join(df_country, by = c("iso_3" = "alpha.3")) 


ERROR: [1m[33mError[39m in `left_join()`:[22m
[1m[22m[33m![39m Join columns in `x` must be present in the data.
[31m✖[39m Problem with `iso_3`.


## SQL

I couldn't access the data file from my computer so this will be fixed later.

## Regex

In the file comments.txt we will find that the data repo contains lines of text, each representing a user comment. Users sometimes include tags in their comments using the format "#tag".

In [12]:
comments <- readLines("https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/comments.txt")  
#Lines are stored in the 'comments' variable

In the code below we created a function that uses str_extract_all to extract all hashtags starting with # from a given comment. We named this function hashtags_extraction. 

When we called this function on the first comment, it successfully extracted the hashtags #programming and #python.

In [13]:
hashtags_extraction <- function(comment) {
  str_extract_all(comment, "#\\w+")[[1]]  
    #Extract all hashtags starting with "#"
}

first_comment_hashtag <- hashtags_extraction(comments[1])
#Extracting hashtags from the first comment in the file 

first_comment_hashtag

[1] "#programming" "#tips"

The following code defines a regular expression regex_python_programming that checks if both the hashtags #programming and #python appear in a comment, regardless of their order. The code also uses str_detect to check if the second comment contains both hashtags. Based on the result, it prints a message indicating whether or not both #programming and #python are mentioned in the comment.

For comment number 2, the output shows that it does not contain both hashtags.

In [20]:
regex_python_programming <- "#programming.*#python|#python.*#programming"
#The '|' operator is used for the or condition, wheter #programming or #python comes first does not matter
#The '.*' allows any characters (or none) between the two hashtags

matches <- str_detect(comments[2], regex_python_programming)
#Check if it contains both hashtags in the second comment

if (matches) {
  print("Both #programming and #python are mentioned in the comment.")
} else {
  print("The comment does not mention both #programming and #python.")
}
#A proper message is printed depending on if both #programming' and #python are found

[1] "The comment does not mention both #programming and #python."
